# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""17035166218277735825""","""17652519963868814543""","""../../../../data/SampleHDS.jso…",6.7933e18,"""10914674418873696274""","""14362335631750911326""",false
"""1534198504717545991""","""10586778608072319122""","""../../../../data/SampleHDS.jso…",1.4592e19,"""10914674418873696274""","""12544608678927418439""",true
"""10772072136811014140""","""13755566295741326223""","""../../../../data/SampleHDS.jso…",1.2568e19,"""1534776865280916198""","""14362335631750911326""",false
"""642657513337944795""","""10586778608072319122""","""../../../../data/SampleHDS.jso…",4.7727e18,"""12147529971680179869""","""14362335631750911326""",false
"""12450727180648882671""","""17652519963868814543""","""../../../../data/SampleHDS.jso…",1.1862e19,"""12147529971680179869""","""10316811931780939832""",true
"""17994206423774747023""","""13755566295741326223""","""../../../../data/SampleHDS.jso…",4.2967e18,"""7916812058305030117""",null,false
"""13666796091799113689""","""13755566295741326223""","""../../../../data/SampleHDS.jso…",3.6049e18,"""12147529971680179869""","""10316811931780939832""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""10421530456688695505""","""1307786959389501633""","""../../../../data/SampleHDS.jso…",9.1389e18,"""FirstMortgage30yr""","""1662383219869985921""","""Rejected"""
"""13057940419739951786""","""8970986323778493984""","""../../../../data/SampleHDS.jso…",1.0326e19,"""FirstMortgage30yr""","""6842710262953036803""","""Accepted"""
"""11146903298179721276""","""9055545976914504551""","""../../../../data/SampleHDS.jso…",1.7199e19,"""MoneyMarketSavingsAccount""","""1662383219869985921""","""Rejected"""
"""9826843087142990640""","""8970986323778493984""","""../../../../data/SampleHDS.jso…",4.5974e18,"""BasicChecking""","""1662383219869985921""","""Rejected"""
"""10707366210877945620""","""1307786959389501633""","""../../../../data/SampleHDS.jso…",7.7153e18,"""BasicChecking""","""14982254092979331165""","""Accepted"""
"""12128385130417276013""","""9055545976914504551""","""../../../../data/SampleHDS.jso…",1.3290e19,"""UPlusFinPersonal""",null,"""Rejected"""
"""284740001734369655""","""9055545976914504551""","""../../../../data/SampleHDS.jso…",8.4425e18,"""BasicChecking""","""14982254092979331165""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""5782417346479323568""","""10934061173546713242""","""../../../../data/SampleHDS.jso…",1.7925e19,"""FirstMortgage30yr""","""16729474100050906583""","""Rejected"""
"""11648524400818304306""","""8728289127192121741""","""../../../../data/SampleHDS.jso…",2.9861e18,"""FirstMortgage30yr""","""4430032388472282973""","""Accepted"""
"""4035700762818031092""","""8635080010971920438""","""../../../../data/SampleHDS.jso…",4.4699e18,"""MoneyMarketSavingsAccount""","""16729474100050906583""","""Rejected"""
"""12417966989149837960""","""8728289127192121741""","""../../../../data/SampleHDS.jso…",1.0495e19,"""BasicChecking""","""16729474100050906583""","""Rejected"""
"""11546184594831846259""","""10934061173546713242""","""../../../../data/SampleHDS.jso…",4.3197e18,"""BasicChecking""","""18141289579815432083""","""Accepted"""
"""1236455152202923613""","""8635080010971920438""","""../../../../data/SampleHDS.jso…",1.5144e19,"""UPlusFinPersonal""",null,"""Rejected"""
"""17769396838642530982""","""8635080010971920438""","""../../../../data/SampleHDS.jso…",1.4045e19,"""BasicChecking""","""18141289579815432083""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
filename=filename
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""5810044136813672070""","""16249872205076037054""",1.4377e19,"""FirstMortgage30yr""","""13256866394841191028""","""Rejected"""
"""17193484032099304275""","""14047044115769636416""",1.0320e18,"""FirstMortgage30yr""","""6022281016663483843""","""Accepted"""
"""7964785004449590286""","""540411473791759681""",1.6790e19,"""MoneyMarketSavingsAccount""","""13256866394841191028""","""Rejected"""
"""10832742311765399231""","""14047044115769636416""",1.3430e19,"""BasicChecking""","""13256866394841191028""","""Rejected"""
"""3983329073171656152""","""16249872205076037054""",5.9512e18,"""BasicChecking""","""1585468701919790330""","""Accepted"""
"""14485207850875705700""","""540411473791759681""",1.7725e19,"""UPlusFinPersonal""",null,"""Rejected"""
"""17438646609208281124""","""540411473791759681""",1.0423e19,"""BasicChecking""","""1585468701919790330""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'